Lớp **Toán ứng dụng và thống kê - 17/11**  
Học kì II, 2019 - 2020  

<hr>

<br>
<center><b><font size="10">ĐỒ ÁN 1</font></b></center>
<br>

<center>
    <font size="4"><b>Nhóm thực hiện:</b></font><br>
    Trần Minh Trí - 1712834&emsp;Nguyễn Nhật Trường - 1712852<br>
    Lê Trường Vĩ - 1712900&emsp;Nguyễn Hửu Trí - 1712832<br>
</center>

<hr>

<br>
<center><b><font size="6">Đánh giá model đã chọn</font></b></center>
<br>

In [1]:
import pandas as pd
import numpy as np 
from sklearn import datasets, linear_model, metrics 
from sklearn.model_selection import train_test_split
import pickle
import os.path

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

import csv
from sklearn.preprocessing import LabelEncoder

In [6]:
def read_data(X_path, Y_path):
    X = pd.read_csv(X_path, index_col='id')
    Y = pd.read_csv(Y_path, index_col='id')
    
    return X, Y

In [3]:
def load_model(filename=None):
    model = pickle.load(open(filename, 'rb'))
    print("Loaded model in: ", filename)
    return model

In [14]:
def model_validation(model, X_test, Y_test):
    prediction = model.predict(X_test)
    mse_out = np.mean((Y_test.values - prediction)**2)
    print("Match: ", model.score(X_test, Y_test)*100, "%")
    print("mse: ", mse_out)
    print("rmse: ",np.sqrt(mse_out))

In [7]:
X_test_path = "X_dummy.csv"
Y_test_path = "Y_dummy.csv"
X, Y = read_data(X_test_path, Y_test_path)

## Model 3 <a class="anchor" id="model-3"></a>

In [8]:
def model_self_labeled_process(X, scale):
    X_train = X.copy()
    X_train.drop(["photos"], axis=1)
    index = [i for i, row in X_train.iterrows() if row.isnull().any()]

    X_train = X_train.dropna()
    X_train['year'] -= min(X['year'])
    
    label_col = ["manufacturer", "model","transmission","color","engineFuel","engineType","bodyType","drivetrain"]
    
    for s in label_col:
        d = scale[s]
        for key in d.keys():
            if key != "Other":
                X_train.replace(key, d[key], inplace=True)
        for u in X_train[s].unique():
            try:
                float(u)
            except ValueError:
                # print("Droping ", u)
                # X_train.replace(u, d["Other"], inplace=True) # Ket qua rat te!
                idx = X_train[ X_train[s] == u ].index
                X_train.drop(idx , inplace=True)
                index.append(idx)
        
    X_train.replace(True, 1, inplace=True)
    X_train.replace(False, 0, inplace=True)
    X_train['x0'] = 1
    
    X_train = X_train.astype(float)
    
    return X_train, index

In [9]:
fname = "model_scaling.pkl"
scale = pickle.load(open(fname, 'rb'))
print("Loaded scale")

Loaded scale


In [10]:
X_test3, drop_idx = model_self_labeled_process(X, scale)
Y_test3 = Y.drop(drop_idx)

X_test3.shape, Y_test3.shape

((6307, 23), (6307, 1))

In [11]:
regressor_3 = load_model("DA1_Model3.pkl")

Loaded model in:  DA1_Model3.pkl


##### Kết quả đánh giá:

In [15]:
model_validation(regressor_3, X_test3, Y_test3)

Match:  78.21774665048886 %
mse:  7744682.260909913
rmse:  2782.926923386583


## Model 4 <a class="anchor" id="model-4"></a>

In [19]:
def transfrom_data(X, pipeline):
    X_ = X.copy()
    X_['x0'] = 1
    return pipeline.transform(X_)

In [20]:
fname = "pipeline.pkl"
full_pipeline = pickle.load(open(fname, 'rb'))
print("Loaded pipeline")

Loaded pipeline


In [22]:
X_test4 = transfrom_data(X, full_pipeline)
X_test4.shape, Y.shape

((6311, 113), (6311, 1))

In [23]:
regressor_4 = load_model("DA1_Model4.pkl")

Loaded model in:  DA1_Model4.pkl


##### Kết quả đánh giá:

In [24]:
model_validation(regressor_4,  X_test4, Y)

Match:  87.22676796737578 %
mse:  4544351.312966027
rmse:  2131.7484169024324


## Model 5 <a class="anchor" id="model-5"></a>

In [25]:
def read_csv(X_path, Y_path):
    x= []
    y= [] 
    with open(X_path,'rt') as f:
        data1 = csv.reader(f)
        for i in data1:
            i.append('1')
            x.append(i)
    with open(Y_path,'rt') as g:
        data2 = csv.reader(g)
        for i in data2:
            y.append(i)
            
    x=np.array(x)[1:,1:]                                         #Xóa dòng đầu cột đầu
    y = np.array(y)[1:,1:].flatten().astype(np.float)            #Xóa dòng đầu cột đầu
    return x, y

In [26]:
X, Y = read_csv(X_test_path, Y_test_path)

In [27]:
def model_validation2(model, X_test, Y_test):
    prediction = model.predict(X_test)
    mse_out = np.mean((Y_test - prediction)**2)
    print("Out-sample match: ", model.score(X_test, Y_test)*100, "%")
    print("Out-sample mse: ", mse_out)
    print("Out-sample rmse: ",np.sqrt(mse_out))

In [28]:
def process_data(X, label_encoder_list, onehot_encoder):
    for i in list(range(0,22)):
        X[:,i] = label_encoder_list[i].transform(X[:,i])
        
    X = onehot_encoder.transform(X)
    
    X = X.astype(np.int)
    
    return X

In [29]:
fname = "encoder.pkl"
encoder = pickle.load(open(fname, 'rb'))
print("Loaded encoder")

Loaded encoder


In [30]:
X_test5 = process_data(X, encoder["label_encoder_list"], encoder["onehot_encoder"])

In [34]:
regressor_5 = load_model("DA1_Model5.pkl")

Loaded model in:  DA1_Model5.pkl


##### Kết quả đánh giá:

In [38]:
prediction = regressor_5.predict(X_test5)
mse_out = np.mean((Y - prediction)**2)
print("Match: ", regressor_5.score(X_test5, Y)*100, "%")
print("mse: ", mse_out)
print("rmse: ",np.sqrt(mse_out))

Match:  90.73110596795786 %
mse:  3297607.8925577975
rmse:  1815.9316871947021
